# Week 9 - Part 2: Widgets and Interactivity

**Course:** Python Data Analysis for Business Intelligence  
**Week:** 9 | **Session:** Wednesday | **Part:** 2 of 3  
**Duration:** 20 minutes | **Date:** June 4, 2025

## Learning Objectives
By the end of this session, you will be able to:
- Master all essential Streamlit widgets for user interaction
- Design intuitive user interfaces for business stakeholders
- Implement data filtering and selection workflows
- Handle user input validation and error management
- Create responsive layouts with columns and containers

---

## 🎯 Business Context: Interactive Business Intelligence

**Real-World Scenario**: You're building a customer analytics dashboard for Olist's operations team. Different stakeholders need different views of the same data:

- **Sales Managers**: Filter by product categories and date ranges
- **Customer Service**: Search by customer ID and satisfaction scores
- **Logistics**: Select delivery regions and shipping methods
- **Marketing**: Choose campaign types and customer segments

**The Challenge**: Create one flexible interface that serves all these needs without overwhelming any single user.

**Today's Solution**: Master Streamlit widgets to build intuitive, role-based filtering systems.

---

## 🛠️ Setup: Environment and Data

Let's start by setting up our development environment and loading real Olist data:

In [1]:
# Standard imports
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

# Course utilities
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# # Setup visualization style
# plt, sns = set_plotting_style()
# setup_colab()

print("✅ Environment setup complete!")
print("🔗 Ready to build interactive dashboards with real business data")

✅ Environment setup complete!
🔗 Ready to build interactive dashboards with real business data


## 📊 Section 1: Core Widget Types (8 minutes)

Let's create a comprehensive widget demonstration app that showcases all the essential Streamlit widgets:

In [2]:
%%writefile widget_showcase_app.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta

# Configure page
st.set_page_config(
    page_title="Streamlit Widget Showcase",
    page_icon="🎛️",
    layout="wide"
)

st.title("🎛️ Streamlit Widget Showcase")
st.markdown("**Interactive elements for business applications**")

# Create sample business data
@st.cache_data
def generate_business_data():
    """
    Generate realistic e-commerce data for widget demonstrations
    """
    np.random.seed(42)
    
    # Date range for last 2 years
    start_date = datetime.now() - timedelta(days=730)
    dates = pd.date_range(start_date, periods=730, freq='D')
    
    data = {
        'date': np.repeat(dates, 3),  # 3 records per day
        'customer_id': [f"CUST_{i:06d}" for i in range(1, len(dates)*3 + 1)],
        'product_category': np.random.choice([
            'Electronics', 'Fashion', 'Home & Garden', 'Books', 'Sports', 
            'Beauty', 'Automotive', 'Toys', 'Health', 'Food'
        ], len(dates)*3),
        'state': np.random.choice([
            'São Paulo', 'Rio de Janeiro', 'Minas Gerais', 'Bahia', 
            'Paraná', 'Rio Grande do Sul', 'Pernambuco', 'Ceará'
        ], len(dates)*3),
        'order_value': np.random.exponential(100, len(dates)*3),
        'customer_satisfaction': np.random.choice([1, 2, 3, 4, 5], len(dates)*3, 
                                                p=[0.05, 0.1, 0.2, 0.35, 0.3]),
        'shipping_method': np.random.choice([
            'Standard', 'Express', 'Premium', 'Economy'
        ], len(dates)*3, p=[0.4, 0.3, 0.2, 0.1]),
        'payment_method': np.random.choice([
            'Credit Card', 'Debit Card', 'Bank Transfer', 'Digital Wallet'
        ], len(dates)*3, p=[0.45, 0.25, 0.2, 0.1])
    }
    
    return pd.DataFrame(data)

# Load data
df = generate_business_data()

# Sidebar for widgets
st.sidebar.header("🎛️ Widget Controls")
st.sidebar.markdown("Explore different widget types and their applications:")

# Section 1: Selection Widgets
st.sidebar.subheader("📝 Selection Widgets")

# 1. Selectbox - Single selection
selected_category = st.sidebar.selectbox(
    "Product Category:",
    options=['All'] + sorted(df['product_category'].unique()),
    help="Choose a product category to filter data"
)

# 2. Multiselect - Multiple selections
selected_states = st.sidebar.multiselect(
    "Brazilian States:",
    options=sorted(df['state'].unique()),
    default=sorted(df['state'].unique())[:3],
    help="Select one or more states for analysis"
)

# 3. Radio - Single choice from visible options
satisfaction_filter = st.sidebar.radio(
    "Customer Satisfaction Filter:",
    options=['All', 'High (4-5)', 'Medium (3)', 'Low (1-2)'],
    help="Filter customers by satisfaction level"
)

# Section 2: Input Widgets
st.sidebar.subheader("📊 Input Widgets")

# 4. Slider - Numeric range
order_value_range = st.sidebar.slider(
    "Order Value Range (R$):",
    min_value=float(df['order_value'].min()),
    max_value=float(df['order_value'].max()),
    value=(50.0, 500.0),
    step=25.0,
    help="Filter orders by value range"
)

# 5. Date input - Date selection
start_date = st.sidebar.date_input(
    "Analysis Start Date:",
    value=df['date'].min().date(),
    min_value=df['date'].min().date(),
    max_value=df['date'].max().date(),
    help="Choose the start date for analysis"
)

end_date = st.sidebar.date_input(
    "Analysis End Date:",
    value=df['date'].max().date(),
    min_value=df['date'].min().date(),
    max_value=df['date'].max().date(),
    help="Choose the end date for analysis"
)

# 6. Number input - Specific numeric input
min_orders = st.sidebar.number_input(
    "Minimum Orders to Display:",
    min_value=1,
    max_value=100,
    value=10,
    step=5,
    help="Set minimum number of orders for category analysis"
)

# Section 3: Interactive Widgets
st.sidebar.subheader("🔄 Interactive Widgets")

# 7. Checkbox - Boolean toggle
show_trends = st.sidebar.checkbox(
    "Show Trend Analysis",
    value=True,
    help="Toggle trend analysis charts"
)

include_weekends = st.sidebar.checkbox(
    "Include Weekend Data",
    value=True,
    help="Include Saturday and Sunday in analysis"
)

# 8. Button - Action trigger
refresh_data = st.sidebar.button(
    "🔄 Refresh Analysis",
    help="Refresh data and regenerate charts"
)

# Main content area
st.markdown("---")

# Apply filters to data
filtered_df = df.copy()

# Apply category filter
if selected_category != 'All':
    filtered_df = filtered_df[filtered_df['product_category'] == selected_category]

# Apply state filter
if selected_states:
    filtered_df = filtered_df[filtered_df['state'].isin(selected_states)]

# Apply satisfaction filter
if satisfaction_filter == 'High (4-5)':
    filtered_df = filtered_df[filtered_df['customer_satisfaction'].isin([4, 5])]
elif satisfaction_filter == 'Medium (3)':
    filtered_df = filtered_df[filtered_df['customer_satisfaction'] == 3]
elif satisfaction_filter == 'Low (1-2)':
    filtered_df = filtered_df[filtered_df['customer_satisfaction'].isin([1, 2])]

# Apply order value filter
filtered_df = filtered_df[
    (filtered_df['order_value'] >= order_value_range[0]) & 
    (filtered_df['order_value'] <= order_value_range[1])
]

# Apply date filter
filtered_df = filtered_df[
    (filtered_df['date'].dt.date >= start_date) & 
    (filtered_df['date'].dt.date <= end_date)
]

# Apply weekend filter
if not include_weekends:
    filtered_df = filtered_df[filtered_df['date'].dt.weekday < 5]

# Display filter results
st.subheader("📈 Filtered Data Analysis")

col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric("Total Orders", f"{len(filtered_df):,}")

with col2:
    avg_value = filtered_df['order_value'].mean()
    st.metric("Average Order Value", f"R$ {avg_value:.2f}")

with col3:
    avg_satisfaction = filtered_df['customer_satisfaction'].mean()
    st.metric("Average Satisfaction", f"{avg_satisfaction:.2f}/5.0")

with col4:
    total_revenue = filtered_df['order_value'].sum()
    st.metric("Total Revenue", f"R$ {total_revenue:,.2f}")

# Charts section
if show_trends and len(filtered_df) > 0:
    st.markdown("---")
    st.subheader("📊 Visual Analysis")
    
    chart_col1, chart_col2 = st.columns(2)
    
    with chart_col1:
        # Category distribution
        category_data = filtered_df['product_category'].value_counts()
        fig = px.pie(
            values=category_data.values,
            names=category_data.index,
            title="Orders by Product Category"
        )
        st.plotly_chart(fig, use_container_width=True)
    
    with chart_col2:
        # State distribution
        state_data = filtered_df['state'].value_counts()
        fig = px.bar(
            x=state_data.index,
            y=state_data.values,
            title="Orders by State",
            labels={'x': 'State', 'y': 'Number of Orders'}
        )
        fig.update_xaxes(tickangle=45)
        st.plotly_chart(fig, use_container_width=True)

# Data preview
st.markdown("---")
st.subheader("📋 Filtered Data Preview")
st.markdown(f"Showing first 10 records from {len(filtered_df):,} total filtered records")

if len(filtered_df) > 0:
    st.dataframe(
        filtered_df.head(10)[[
            'date', 'customer_id', 'product_category', 'state',
            'order_value', 'customer_satisfaction'
        ]],
        use_container_width=True
    )
else:
    st.warning("⚠️ No data matches your current filter criteria. Try adjusting the filters.")

# Widget status display
st.markdown("---")
st.subheader("🔍 Current Filter Status")

status_col1, status_col2 = st.columns(2)

with status_col1:
    st.markdown("**Active Filters:**")
    st.write(f"• Category: {selected_category}")
    st.write(f"• States: {', '.join(selected_states) if selected_states else 'None'}")
    st.write(f"• Satisfaction: {satisfaction_filter}")
    st.write(f"• Order Value: R$ {order_value_range[0]:.0f} - R$ {order_value_range[1]:.0f}")

with status_col2:
    st.markdown("**Analysis Settings:**")
    st.write(f"• Date Range: {start_date} to {end_date}")
    st.write(f"• Show Trends: {'Yes' if show_trends else 'No'}")
    st.write(f"• Include Weekends: {'Yes' if include_weekends else 'No'}")
    st.write(f"• Minimum Orders: {min_orders}")

# Footer
st.markdown("---")
st.markdown(
    "**🎛️ Widget Demo** | "
    "Built with Streamlit | "
    "**📊 Data:** Simulated E-commerce | "
    f"**🕒 Last Updated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
)

Writing widget_showcase_app.py


## 🎯 Section 2: Advanced Widget Patterns (6 minutes)

Now let's explore advanced widget patterns commonly used in business applications:

In [3]:
%%writefile advanced_widgets_app.py

import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Configure page
st.set_page_config(
    page_title="Advanced Widget Patterns",
    page_icon="⚡",
    layout="wide"
)

st.title("⚡ Advanced Widget Patterns for Business Apps")
st.markdown("**Professional patterns for complex business requirements**")

# Pattern 1: Conditional Widget Display
st.header("🔀 Pattern 1: Conditional Widget Display")
st.markdown("Show/hide widgets based on user selections - common in role-based dashboards.")

with st.container():
    col1, col2 = st.columns([1, 2])
    
    with col1:
        user_role = st.selectbox(
            "Select Your Role:",
            ['Sales Manager', 'Marketing Analyst', 'Operations Director', 'Finance Controller']
        )
    
    with col2:
        # Show different widgets based on role
        if user_role == 'Sales Manager':
            st.info("🎯 Sales-focused controls active")
            territory = st.selectbox("Sales Territory:", ['North', 'South', 'East', 'West', 'Central'])
            quota_period = st.selectbox("Quota Period:", ['Monthly', 'Quarterly', 'Annual'])
            
        elif user_role == 'Marketing Analyst':
            st.info("📢 Marketing analytics controls active")
            campaign_type = st.multiselect(
                "Campaign Types:", 
                ['Email', 'Social Media', 'Search Ads', 'Display', 'Influencer']
            )
            attribution_window = st.slider("Attribution Window (days):", 1, 30, 7)
            
        elif user_role == 'Operations Director':
            st.info("⚙️ Operations monitoring controls active")
            warehouse = st.selectbox("Warehouse:", ['São Paulo', 'Rio', 'Belo Horizonte', 'Salvador'])
            sla_metric = st.radio("SLA Metric:", ['Delivery Time', 'Order Accuracy', 'Return Rate'])
            
        elif user_role == 'Finance Controller':
            st.info("💰 Financial controls active")
            currency = st.selectbox("Currency:", ['BRL', 'USD', 'EUR'])
            fiscal_period = st.date_input("Fiscal Period Start:", datetime.now().replace(month=1, day=1))

# Pattern 2: Dynamic Widget Updates
st.header("🔄 Pattern 2: Dynamic Widget Updates")
st.markdown("Widget options change based on other selections - cascading dropdowns.")

with st.container():
    col1, col2, col3 = st.columns(3)
    
    # Sample hierarchical data
    location_data = {
        'Brazil': {
            'Southeast': ['São Paulo', 'Rio de Janeiro', 'Minas Gerais', 'Espírito Santo'],
            'South': ['Rio Grande do Sul', 'Santa Catarina', 'Paraná'],
            'Northeast': ['Bahia', 'Pernambuco', 'Ceará', 'Paraíba']
        }
    }
    
    with col1:
        country = st.selectbox("Country:", list(location_data.keys()))
    
    with col2:
        if country:
            regions = list(location_data[country].keys())
            region = st.selectbox("Region:", regions)
        else:
            region = None
    
    with col3:
        if region:
            states = location_data[country][region]
            state = st.selectbox("State:", states)
        else:
            state = None
    
    if state:
        st.success(f"✅ Selected: {state}, {region}, {country}")

# Pattern 3: Form-based Input with Validation
st.header("📝 Pattern 3: Form-based Input with Validation")
st.markdown("Professional form handling with validation - essential for data entry applications.")

with st.form("business_form"):
    st.subheader("Customer Order Form")
    
    form_col1, form_col2 = st.columns(2)
    
    with form_col1:
        customer_name = st.text_input(
            "Customer Name *",
            placeholder="Enter full name",
            help="Required field"
        )
        
        customer_email = st.text_input(
            "Email Address *",
            placeholder="customer@email.com",
            help="Valid email required"
        )
        
        order_value = st.number_input(
            "Order Value (R$) *",
            min_value=0.01,
            max_value=10000.00,
            value=100.00,
            step=0.01,
            help="Minimum R$ 0.01"
        )
    
    with form_col2:
        product_category = st.selectbox(
            "Product Category *",
            ['', 'Electronics', 'Fashion', 'Home & Garden', 'Books', 'Sports']
        )
        
        shipping_method = st.radio(
            "Shipping Method:",
            ['Standard (5-7 days)', 'Express (2-3 days)', 'Premium (1-2 days)']
        )
        
        special_instructions = st.text_area(
            "Special Instructions:",
            placeholder="Any special delivery instructions...",
            height=100
        )
    
    # Form submission with validation
    submitted = st.form_submit_button("📦 Submit Order", type="primary")
    
    if submitted:
        # Validation logic
        errors = []
        
        if not customer_name.strip():
            errors.append("Customer name is required")
        
        if not customer_email.strip() or '@' not in customer_email:
            errors.append("Valid email address is required")
        
        if not product_category:
            errors.append("Product category must be selected")
        
        if order_value <= 0:
            errors.append("Order value must be greater than zero")
        
        # Display results
        if errors:
            st.error("❌ Please fix the following errors:")
            for error in errors:
                st.error(f"• {error}")
        else:
            st.success("✅ Order submitted successfully!")
            st.info(f"Order ID: ORD-{datetime.now().strftime('%Y%m%d%H%M%S')}")
            
            # Display order summary
            with st.expander("📋 Order Summary", expanded=True):
                st.write(f"**Customer:** {customer_name}")
                st.write(f"**Email:** {customer_email}")
                st.write(f"**Category:** {product_category}")
                st.write(f"**Value:** R$ {order_value:.2f}")
                st.write(f"**Shipping:** {shipping_method}")
                if special_instructions:
                    st.write(f"**Instructions:** {special_instructions}")

# Pattern 4: Session State Management
st.header("💾 Pattern 4: Session State Management")
st.markdown("Maintain state across widget interactions - essential for multi-step processes.")

# Initialize session state
if 'counter' not in st.session_state:
    st.session_state.counter = 0
if 'user_data' not in st.session_state:
    st.session_state.user_data = []

with st.container():
    col1, col2, col3 = st.columns(3)
    
    with col1:
        if st.button("➕ Increment Counter"):
            st.session_state.counter += 1
    
    with col2:
        if st.button("➖ Decrement Counter"):
            st.session_state.counter -= 1
    
    with col3:
        if st.button("🔄 Reset Counter"):
            st.session_state.counter = 0
    
    st.metric("Current Counter Value", st.session_state.counter)
    
    # Data collection example
    st.markdown("**Add Data Points:**")
    new_value = st.number_input("Enter a value:", value=0.0)
    
    if st.button("📊 Add to Dataset"):
        st.session_state.user_data.append({
            'timestamp': datetime.now(),
            'value': new_value,
            'counter_state': st.session_state.counter
        })
        st.success(f"Added value {new_value} to dataset")
    
    if st.session_state.user_data:
        st.markdown("**Current Dataset:**")
        df_session = pd.DataFrame(st.session_state.user_data)
        st.dataframe(df_session, use_container_width=True)
        
        if st.button("🗑️ Clear Dataset"):
            st.session_state.user_data = []
            st.rerun()

# Footer
st.markdown("---")
st.markdown(
    "**⚡ Advanced Patterns** | "
    "Professional widget patterns for business applications | "
    "**📚 Next:** Data visualization integration"
)

Writing advanced_widgets_app.py


## 🎨 Section 3: Layout and Styling (6 minutes)

Professional appearance is crucial for business applications. Let's explore layout and styling options:

In [4]:
%%writefile layout_styling_app.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# Configure page with custom styling
st.set_page_config(
    page_title="Professional Layouts",
    page_icon="🎨",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for professional appearance
st.markdown("""
<style>
    .main-header {
        background: linear-gradient(90deg, #1e3c72, #2a5298);
        padding: 2rem;
        border-radius: 10px;
        color: white;
        margin-bottom: 2rem;
    }
    
    .metric-container {
        background: #f8f9fa;
        padding: 1.5rem;
        border-radius: 8px;
        border-left: 4px solid #007bff;
        margin: 1rem 0;
    }
    
    .status-success {
        background: #d4edda;
        border: 1px solid #c3e6cb;
        color: #155724;
        padding: 1rem;
        border-radius: 5px;
    }
    
    .status-warning {
        background: #fff3cd;
        border: 1px solid #ffeaa7;
        color: #856404;
        padding: 1rem;
        border-radius: 5px;
    }
</style>
""", unsafe_allow_html=True)

# Professional header
st.markdown("""
<div class="main-header">
    <h1>🎨 Professional Layout & Styling</h1>
    <p>Enterprise-grade dashboard design patterns for business applications</p>
</div>
""", unsafe_allow_html=True)

# Sample data
@st.cache_data
def load_sample_metrics():
    return {
        'revenue': 1250000,
        'growth': 15.3,
        'customers': 45230,
        'satisfaction': 4.2,
        'orders': 12450,
        'conversion': 3.8
    }

metrics = load_sample_metrics()

# Executive summary section
st.subheader("📊 Executive Summary")

# Multi-column layout for KPIs
kpi_col1, kpi_col2, kpi_col3, kpi_col4, kpi_col5, kpi_col6 = st.columns(6)

with kpi_col1:
    st.metric(
        label="💰 Revenue",
        value=f"R$ {metrics['revenue']:,.0f}",
        delta=f"+{metrics['growth']:.1f}%"
    )

with kpi_col2:
    st.metric(
        label="👥 Customers",
        value=f"{metrics['customers']:,}",
        delta="+1,234"
    )

with kpi_col3:
    st.metric(
        label="📦 Orders",
        value=f"{metrics['orders']:,}",
        delta="+567"
    )

with kpi_col4:
    st.metric(
        label="⭐ Satisfaction",
        value=f"{metrics['satisfaction']:.1f}/5.0",
        delta="+0.2"
    )

with kpi_col5:
    st.metric(
        label="🎯 Conversion",
        value=f"{metrics['conversion']:.1f}%",
        delta="+0.3%"
    )

with kpi_col6:
    # Status indicator
    if metrics['satisfaction'] > 4.0:
        st.markdown("""
        <div class="status-success">
            <strong>✅ System Status</strong><br>
            All systems operational
        </div>
        """, unsafe_allow_html=True)
    else:
        st.markdown("""
        <div class="status-warning">
            <strong>⚠️ System Status</strong><br>
            Attention required
        </div>
        """, unsafe_allow_html=True)

# Tabbed interface for different views
st.markdown("---")
tab1, tab2, tab3, tab4 = st.tabs(["📈 Performance", "🎯 Marketing", "⚙️ Operations", "💰 Finance"])

with tab1:
    st.subheader("Performance Analytics")
    
    # Two-column layout for charts
    perf_col1, perf_col2 = st.columns(2)
    
    with perf_col1:
        # Generate sample time series data
        dates = pd.date_range('2024-01-01', periods=12, freq='M')
        revenue_data = np.random.normal(100000, 15000, 12).cumsum()
        
        fig = px.line(
            x=dates, 
            y=revenue_data,
            title="Monthly Revenue Trend",
            labels={'x': 'Month', 'y': 'Revenue (R$)'}
        )
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)
    
    with perf_col2:
        # Customer acquisition data
        channels = ['Organic', 'Paid Search', 'Social Media', 'Email', 'Direct']
        acquisitions = [1250, 890, 650, 420, 380]
        
        fig = px.bar(
            x=channels,
            y=acquisitions,
            title="Customer Acquisition by Channel",
            labels={'x': 'Channel', 'y': 'New Customers'}
        )
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

with tab2:
    st.subheader("Marketing Performance")
    
    # Campaign performance table
    campaign_data = pd.DataFrame({
        'Campaign': ['Summer Sale', 'Back to School', 'Holiday Special', 'New Year Promo'],
        'Impressions': [125000, 98000, 156000, 87000],
        'Clicks': [3750, 2940, 4680, 2610],
        'Conversions': [187, 147, 234, 130],
        'Cost (R$)': [12500, 9800, 15600, 8700],
        'ROI (%)': [15.2, 12.8, 18.7, 14.3]
    })
    
    st.dataframe(
        campaign_data.style.format({
            'Impressions': '{:,}',
            'Clicks': '{:,}',
            'Cost (R$)': 'R$ {:,.0f}',
            'ROI (%)': '{:.1f}%'
        }),
        use_container_width=True
    )
    
    # Channel effectiveness
    marketing_col1, marketing_col2 = st.columns(2)
    
    with marketing_col1:
        fig = px.pie(
            values=campaign_data['Conversions'],
            names=campaign_data['Campaign'],
            title="Conversions by Campaign"
        )
        st.plotly_chart(fig, use_container_width=True)
    
    with marketing_col2:
        fig = px.scatter(
            campaign_data,
            x='Cost (R$)',
            y='Conversions',
            size='Impressions',
            color='ROI (%)',
            title="Campaign Cost vs. Conversions",
            hover_data=['Campaign']
        )
        st.plotly_chart(fig, use_container_width=True)

with tab3:
    st.subheader("Operations Dashboard")
    
    # Operational metrics in expandable sections
    with st.expander("📦 Order Processing Metrics", expanded=True):
        ops_col1, ops_col2, ops_col3 = st.columns(3)
        
        with ops_col1:
            st.metric("Orders Today", "1,247", "+12%")
            st.metric("Processing Time", "2.3 hrs", "-0.5 hrs")
        
        with ops_col2:
            st.metric("Fulfillment Rate", "98.7%", "+1.2%")
            st.metric("Return Rate", "2.1%", "-0.3%")
        
        with ops_col3:
            st.metric("Customer Issues", "23", "-5")
            st.metric("Resolution Time", "4.2 hrs", "-1.1 hrs")
    
    with st.expander("🚚 Logistics Performance"):
        # Delivery performance chart
        delivery_data = pd.DataFrame({
            'Region': ['São Paulo', 'Rio de Janeiro', 'Minas Gerais', 'Bahia', 'Paraná'],
            'On-time Delivery (%)': [94.2, 91.5, 89.8, 87.3, 92.1],
            'Average Delivery Time (days)': [2.1, 2.8, 3.2, 4.1, 2.5]
        })
        
        fig = px.bar(
            delivery_data,
            x='Region',
            y='On-time Delivery (%)',
            title="On-time Delivery by Region",
            color='Average Delivery Time (days)',
            color_continuous_scale='RdYlGn_r'
        )
        st.plotly_chart(fig, use_container_width=True)

with tab4:
    st.subheader("Financial Overview")
    
    # Financial summary cards
    fin_col1, fin_col2 = st.columns(2)
    
    with fin_col1:
        st.markdown("""
        <div class="metric-container">
            <h4>💰 Revenue Summary</h4>
            <p><strong>This Month:</strong> R$ 1,250,000</p>
            <p><strong>Last Month:</strong> R$ 1,087,000</p>
            <p><strong>Growth:</strong> +15.0% 📈</p>
            <p><strong>YTD Total:</strong> R$ 12,750,000</p>
        </div>
        """, unsafe_allow_html=True)
    
    with fin_col2:
        st.markdown("""
        <div class="metric-container">
            <h4>💳 Payment Analysis</h4>
            <p><strong>Credit Cards:</strong> 68.5%</p>
            <p><strong>Debit Cards:</strong> 18.2%</p>
            <p><strong>Bank Transfer:</strong> 9.8%</p>
            <p><strong>Digital Wallet:</strong> 3.5%</p>
        </div>
        """, unsafe_allow_html=True)
    
    # Cost breakdown
    cost_data = pd.DataFrame({
        'Category': ['Product Costs', 'Marketing', 'Operations', 'Technology', 'Personnel'],
        'Amount': [650000, 125000, 89000, 67000, 234000],
        'Budget': [680000, 130000, 95000, 70000, 240000]
    })
    
    fig = px.bar(
        cost_data,
        x='Category',
        y=['Amount', 'Budget'],
        title="Actual vs. Budget Spending",
        barmode='group'
    )
    st.plotly_chart(fig, use_container_width=True)

# Sidebar configuration panel
with st.sidebar:
    st.header("🎛️ Dashboard Controls")
    
    # Theme selection
    theme = st.selectbox(
        "Dashboard Theme:",
        ['Professional Blue', 'Corporate Gray', 'Modern Dark', 'Clean White']
    )
    
    # Refresh settings
    auto_refresh = st.checkbox("Auto-refresh data", value=True)
    refresh_interval = st.slider("Refresh interval (minutes)", 1, 60, 5)
    
    # Export options
    st.markdown("---")
    st.subheader("📊 Export Options")
    
    if st.button("📄 Export PDF Report"):
        st.success("PDF report generated!")
    
    if st.button("📊 Export Excel Data"):
        st.success("Excel file downloaded!")
    
    if st.button("📧 Email Summary"):
        st.success("Summary emailed to stakeholders!")
    
    # User preferences
    st.markdown("---")
    st.subheader("👤 User Preferences")
    
    notifications = st.checkbox("Enable notifications", value=True)
    show_tooltips = st.checkbox("Show help tooltips", value=True)
    compact_view = st.checkbox("Compact view mode", value=False)

# Footer with professional styling
st.markdown("---")
st.markdown("""
<div style="text-align: center; color: #666; padding: 2rem;">
    <p><strong>🏢 Olist Business Intelligence Platform</strong></p>
    <p>Built with Streamlit | Data updated in real-time | 
    <a href="#" style="color: #007bff;">Support</a> | 
    <a href="#" style="color: #007bff;">Documentation</a></p>
</div>
""", unsafe_allow_html=True)

Writing layout_styling_app.py


## 🎯 Key Takeaways

✅ **Widget Mastery**: All essential widgets for business applications  
✅ **Advanced Patterns**: Conditional displays, forms, and validation  
✅ **Professional Layouts**: Multi-column, tabs, and expandable sections  
✅ **Session State**: Maintaining user interactions across re-runs  
✅ **Business Focus**: Role-based interfaces and stakeholder-friendly design  

## 🔜 Coming Up in Part 3

In the final session today, we'll integrate advanced data visualization with Streamlit and connect to live Supabase data for real-time business dashboards.

**Preview Topics:**
- Plotly integration for interactive charts
- Real-time data updates with caching
- Supabase connection setup
- Performance optimization techniques

---

## 💼 Practice Exercise

**Scenario**: Create a customer service dashboard with role-based filtering for the Olist customer support team.

**Requirements**:
1. User role selection (Manager, Agent, Analyst)
2. Conditional widgets based on role
3. Form for logging customer interactions
4. Session state to track daily statistics

**Time**: 10 minutes

---

*Next: [Part 3 - Data Visualization →](01_streamlit_fundamentals_part3_data_visualization.ipynb)*